In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns

import sys
sys.path.append('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/src/')
import _config

sys.path.append('/home/unix/maxwshen/')
from mylib import util

notebook_nm = 'fig_risingstar-ill_rl100'
results_dir = _config.RESULTS_PLACE + f'{notebook_nm}/'
util.ensure_dir_exists(results_dir)

Using data folder:
 /ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/data/


In [2]:
## Load design df

modelexp_df = pd.read_csv(_config.DATA_DIR + f'modelexp_ill_risingstar_rs.csv')
datagroup_df = pd.read_csv(_config.DATA_DIR + f'datagroup_ill_risingstar.csv')

modelexp_df = modelexp_df.merge(datagroup_df, on = 'dataset')

modelexp_df.head()

,random_seed,dataset,Name,datasource,Unnamed: 0,data_readlen,data_num_proposed_gts,data_risingstar_num,data_proposal_type,data_num_groups
0,0,ill_rl100_rs5,0,illumina_full,0,100,2,5,smart,10
1,1,ill_rl100_rs5,29,illumina_full,0,100,2,5,smart,10
2,2,ill_rl100_rs5,58,illumina_full,0,100,2,5,smart,10
3,3,ill_rl100_rs5,87,illumina_full,0,100,2,5,smart,10
4,4,ill_rl100_rs5,116,illumina_full,0,100,2,5,smart,10


In [3]:
## Load ground truth data

fitness_df = pd.read_csv('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_pt/fullgt_fitness.csv', index_col = 0)
fitness_df.to_csv(results_dir + f'ground_truth_fitness.csv')
fitness_df.head()

fq_df = pd.read_csv('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/pb_e_form_dataset/badran_pacbio_pivot_1pct.csv')
time_cols = [col for col in fq_df.columns if 'hrs' in col]
fq_df = fq_df.rename(columns = {col: str(idx) for idx, col in enumerate(time_cols)})
fq_df = fq_df.set_index('Abbrev genotype')

In [4]:
fq_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
Abbrev genotype,,,,,,,,,,,,,,,,,,,,,
VIWS.DNGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003802,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.101351,0.806228,0.800000
VIW..DNGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.023715,0.006780,0.000000,0.067568,0.031142,0.042424
VIW...N.ERI.YC.KSKL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.102767,0.569492,0.566372,0.067568,0.000000,0.030303
VIWS..NGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.003953,0.003390,0.000000,0.135135,0.128028,0.024242
VIWS.DNGE.I.YC.KSKL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.018182


In [5]:
int_tp = 16

cols = [str(s) for s in range(int_tp + 1)]
keep_gts = list(fq_df[fq_df[cols].apply(sum, axis = 'columns') > 0.01].index)
keep_gts

['...................',
 'V...........YC.....',
 'V............C.....',
 '............YC.....',
 'V..................',
 'V...........Y......',
 'V.........I.YC.....',
 'V..........PYC.....',
 'V...........YC.K...',
 'VI..........YC.....',
 '............Y......',
 'VI.................',
 'V..S........YC.....',
 'V....D......YC.....']

## Evaluate single prediction

In [12]:
row = modelexp_df[modelexp_df['Name'] == 26].iloc[0]
print(row)

model_fold = '/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_from_reads_pt_multi/modelexp_ill_risingstar_rs/'
model_nm = row['Name']

pred_fitness_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_fitness.csv', index_col = 0)
pred_fq_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_genotype_matrix.csv', index_col = 0)

last_t_col = pred_fq_df.columns[-1]
pfqdfs = pred_fq_df.sort_values(by = last_t_col, ascending = False)
consensus_gt = pfqdfs.iloc[0].name
consensus_fq = pfqdfs[last_t_col].iloc[0]

pred_consensus_fitness = pred_fitness_df[pred_fitness_df['Genotype'] == consensus_gt]['Inferred fitness'].iloc[0]

# Predict consensus gt as rising star if fq < 0.50
second_last_t_col = pred_fq_df.columns[-2]
is_rising = pred_fq_df.loc[consensus_gt, last_t_col] - pred_fq_df.loc[consensus_gt, second_last_t_col]
not_consensus = bool(pred_fq_df.loc[consensus_gt, last_t_col] < 0.50)
if is_rising and not_consensus:
    pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] >= pred_consensus_fitness)
else:
    pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] > pred_consensus_fitness)

# Get observed rising stars
last_t_col = pred_fq_df.columns[-1]
second_last_t_col = pred_fq_df.columns[-2]
print()

ofqdfs = fq_df.sort_values(by = last_t_col, ascending = False)
obs_consensus_gt = ofqdfs.iloc[0].name
obs_is_rising = fq_df.loc[obs_consensus_gt, last_t_col] - fq_df.loc[obs_consensus_gt, second_last_t_col]
obsnot_consensus = bool(fq_df.loc[obs_consensus_gt, last_t_col] < 0.50)

obs_consensus_fitness = fitness_df[fitness_df['Genotype'] == obs_consensus_gt]['Fitness'].iloc[0]
int_tp = row['data_risingstar_num']

# Rising in last two timepoints
crit = (fq_df[str(int_tp)] - fq_df[str(int_tp - 1)] > 0)
keep_gts_crit3 = set(fq_df[crit].index)

# Above 3% frequency in last timepoint
last_timepoint_min_fq = 0.03
keep_gts_crit4 = set(fq_df[fq_df[str(int_tp)] >= last_timepoint_min_fq].index)

keep_gts = keep_gts_crit3 & keep_gts_crit4
crit = (fitness_df['Genotype'].isin(keep_gts))
dfs = fitness_df[crit]

# Label most frequent gt as rising star if fq < 0.50 and fq > 0.50 later
if obs_is_rising and obsnot_consensus:
    dfs['Rising star, observed'] = (dfs['Fitness'] >= obs_consensus_fitness)
else:
    dfs['Rising star, observed'] = (dfs['Fitness'] > obs_consensus_fitness)
        
        
pred_df = pred_fitness_df.merge(dfs, on = 'Genotype', how = 'outer')
pred_df['Rising star, observed'] = pred_df['Rising star, observed'].fillna(value = False)
pred_df['Rising star, predicted'] = pred_df['Rising star, predicted'].fillna(value = False)
display(pred_df)

random_seed                           0
dataset                  ill_rl100_rs31
Name                                 26
datasource                illumina_full
Unnamed: 0                           26
data_readlen                        100
data_num_proposed_gts                56
data_risingstar_num                  31
data_proposal_type                smart
data_num_groups                      10
Name: 260, dtype: object



/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Inferred fitness,"Rising star, predicted",Fitness,"Rising star, observed"
0,V..................,0.134257,False,NaN,False
1,VIW...NGE.I.YC.KS.L,10.995110,False,3.345836,False
2,VI.................,0.158356,False,NaN,False
3,...................,0.087454,False,NaN,False
4,...SG.N.E.I.YC....L,0.200798,False,NaN,False
5,VI......E.I.YCD....,1.997056,False,NaN,False
6,V.....N...I.YC.K..L,0.977789,False,NaN,False
7,VI.....GE.I.YCDKS.L,14.750859,False,NaN,False
8,VIW...N....PYCDKS.L,0.555595,False,NaN,False
9,VIW...N.....YC.KS.L,0.567772,False,NaN,False


In [7]:
# AUROC, AUPRC

pred_df['Inferred fitness'] = pred_df['Inferred fitness'].fillna(value = 0)

from sklearn.metrics import roc_auc_score
auroc = roc_auc_score(
    pred_df['Rising star, observed'],
    pred_df['Inferred fitness'],
)
print(auroc)

from sklearn.metrics import average_precision_score
aps = average_precision_score(
    pred_df['Rising star, observed'],
    pred_df['Inferred fitness'],
)
print(aps)

0.15238095238095237
0.16813186813186815


## evaluate many predictions

In [8]:
model_fold = '/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_from_reads_pt_multi/modelexp_ill_risingstar_rs/'

from collections import defaultdict
stats_dd = defaultdict(list)
for idx, row in modelexp_df[modelexp_df['random_seed'] == 0].iterrows():
    model_nm = row['Name']

    pred_fitness_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_fitness.csv', index_col = 0)
    pred_fq_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_genotype_matrix.csv', index_col = 0)

    last_t_col = pred_fq_df.columns[-1]
    pfqdfs = pred_fq_df.sort_values(by = last_t_col, ascending = False)
    consensus_gt = pfqdfs.iloc[0].name
    consensus_fq = pfqdfs[last_t_col].iloc[0]

    pred_consensus_fitness = pred_fitness_df[pred_fitness_df['Genotype'] == consensus_gt]['Inferred fitness'].iloc[0]

    # Predict consensus gt as rising star if fq < 0.50
    second_last_t_col = pred_fq_df.columns[-2]
    is_rising = pred_fq_df.loc[consensus_gt, last_t_col] - pred_fq_df.loc[consensus_gt, second_last_t_col]
    not_consensus = bool(pred_fq_df.loc[consensus_gt, last_t_col] < 0.50)
    if is_rising and not_consensus:
        pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] >= pred_consensus_fitness)
    else:
        pred_fitness_df['Rising star, predicted'] = (pred_fitness_df['Inferred fitness'] > pred_consensus_fitness)


        
    # Get observed rising stars
    last_t_col = pred_fq_df.columns[-1]
    second_last_t_col = pred_fq_df.columns[-2]

    ofqdfs = fq_df.sort_values(by = last_t_col, ascending = False)
    obs_consensus_gt = ofqdfs.iloc[0].name
    obs_is_rising = fq_df.loc[obs_consensus_gt, last_t_col] - fq_df.loc[obs_consensus_gt, second_last_t_col]
    obsnot_consensus = bool(fq_df.loc[obs_consensus_gt, last_t_col] < 0.50)

    obs_consensus_fitness = fitness_df[fitness_df['Genotype'] == obs_consensus_gt]['Fitness'].iloc[0]
    int_tp = row['data_risingstar_num']

    # Rising in last two timepoints
    crit = (fq_df[str(int_tp)] - fq_df[str(int_tp - 1)] > 0)
    keep_gts_crit3 = set(fq_df[crit].index)

    # Above 3% frequency in last timepoint
    last_timepoint_min_fq = 0.03
    keep_gts_crit4 = set(fq_df[fq_df[str(int_tp)] >= last_timepoint_min_fq].index)

    keep_gts = keep_gts_crit3 & keep_gts_crit4
    crit = (fitness_df['Genotype'].isin(keep_gts))
    dfs = fitness_df[crit]

    # Label most frequent gt as rising star if fq < 0.50 and fq > 0.50 later
    if obs_is_rising and obsnot_consensus:
        dfs['Rising star, observed'] = (dfs['Fitness'] >= obs_consensus_fitness)
    else:
        dfs['Rising star, observed'] = (dfs['Fitness'] > obs_consensus_fitness)

        
    # Combine
    pred_df = pred_fitness_df.merge(dfs, on = 'Genotype', how = 'outer')
    pred_df['Rising star, observed'] = pred_df['Rising star, observed'].fillna(value = False)
    pred_df['Rising star, predicted'] = pred_df['Rising star, predicted'].fillna(value = False)
    
    
    # Record stats
    stats_dd['random_seed'].append(row['random_seed'])
    stats_dd['data_risingstar_num'].append(row['data_risingstar_num'])
    stats_dd['Num pred rising stars'].append(sum(pred_df['Rising star, predicted']))
    stats_dd['Num true rising stars'].append(sum(pred_df['Rising star, observed']))
    
    crit = (pred_df['Rising star, predicted'] == True) & (pred_df['Rising star, observed'] == True)
    stats_dd['True positive'].append(sum(crit))

    crit = (pred_df['Rising star, predicted'] == True) & (pred_df['Rising star, observed'] == False)
    stats_dd['False positive'].append(sum(crit))
    
    crit = (pred_df['Rising star, predicted'] == False) & (pred_df['Rising star, observed'] == False)
    stats_dd['True negative'].append(sum(crit))
    
    crit = (pred_df['Rising star, predicted'] == False) & (pred_df['Rising star, observed'] == True)
    stats_dd['False negative'].append(sum(crit))
    
    # AUROC, AUPRC
    pred_df['Inferred fitness'] = pred_df['Inferred fitness'].fillna(value = 0)

    from sklearn.metrics import roc_auc_score
    try:
        auroc = roc_auc_score(
            pred_df['Rising star, observed'],
            pred_df['Inferred fitness'],
        )
    except ValueError:
        auroc = np.nan
    stats_dd['AUROC'].append(auroc)

    from sklearn.metrics import average_precision_score
    try:
        aps = average_precision_score(
            pred_df['Rising star, observed'],
            pred_df['Inferred fitness'],
        )
    except ValueError:
        auroc = np.nan
    stats_dd['Average precision score'].append(aps)

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.p

In [9]:
stats_df = pd.DataFrame(stats_dd)
stats_df['Sensitivity'] = stats_df['True positive'] / (stats_df['True positive'] + stats_df['False negative'])
stats_df['Specificity'] = stats_df['True negative'] / (stats_df['True negative'] + stats_df['False positive'])

stats_df['Precision'] = stats_df['True positive'] / (stats_df['True positive'] + stats_df['False positive'])
stats_df['Recall'] = stats_df['True positive'] / (stats_df['True positive'] + stats_df['False negative'])

stats_df.shape

(29, 14)

In [10]:
dfs = stats_df[stats_df['random_seed'] == 0]
dfs

,random_seed,data_risingstar_num,Num pred rising stars,Num true rising stars,True positive,False positive,True negative,False negative,AUROC,Average precision score,Sensitivity,Specificity,Precision,Recall
0,0,5,1,1,1,0,1,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0,6,1,1,1,0,1,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,0,7,1,1,1,0,1,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,0,8,3,1,1,2,1,0,0.333333,0.333333,1.000000,0.333333,0.333333,1.000000
4,0,9,4,5,2,2,1,3,0.400000,0.775000,0.400000,0.333333,0.500000,0.400000
5,0,10,0,0,0,0,5,0,NaN,NaN,NaN,1.000000,NaN,NaN
6,0,11,0,0,0,0,6,0,NaN,NaN,NaN,1.000000,NaN,NaN
7,0,12,0,0,0,0,5,0,NaN,NaN,NaN,1.000000,NaN,NaN
8,0,13,0,0,0,0,5,0,NaN,NaN,NaN,1.000000,NaN,NaN
9,0,14,0,0,0,0,5,0,NaN,NaN,NaN,1.000000,NaN,NaN


In [11]:
dfs[['Precision', 'Recall', 'AUROC', 'Average precision score']].describe()

,Precision,Recall,AUROC,Average precision score
count,17.000000,17.000000,17.000000,17.000000
mean,0.509804,0.466667,0.568431,0.534649
std,0.418623,0.408078,0.313903,0.321461
min,0.000000,0.000000,0.041667,0.142857
25%,0.000000,0.000000,0.333333,0.291667
50%,0.500000,0.400000,0.577381,0.387320
75%,1.000000,1.000000,0.800000,0.775000
max,1.000000,1.000000,1.000000,1.000000


## obs rising stars

In [9]:
model_fold = '/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_from_reads_pt_multi/modelexp_ill_risingstar_rs/'

from collections import defaultdict
for idx, row in modelexp_df[modelexp_df['random_seed'] == 0].iterrows():
    model_nm = row['Name']

    pred_fitness_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_fitness.csv', index_col = 0)
    pred_fq_df = pd.read_csv(f'{model_fold}/model_{model_nm}/_final_genotype_matrix.csv', index_col = 0)

    last_t_col = pred_fq_df.columns[-1]
    second_last_t_col = pred_fq_df.columns[-2]

    ofqdfs = fq_df.sort_values(by = last_t_col, ascending = False)
    obs_consensus_gt = ofqdfs.iloc[0].name
    obs_is_rising = fq_df.loc[obs_consensus_gt, last_t_col] - fq_df.loc[obs_consensus_gt, second_last_t_col]
    obsnot_consensus = bool(fq_df.loc[obs_consensus_gt, last_t_col] < 0.50)
    
    obs_consensus_fitness = fitness_df[fitness_df['Genotype'] == obs_consensus_gt]['Fitness'].iloc[0]

#     # Cumulative frequency above 1%
    int_tp = row['data_risingstar_num']
#     cols = [str(s) for s in range(int_tp + 1)]
#     keep_gts_crit1 = set(fq_df[fq_df[cols].apply(sum, axis = 'columns') > 0.01].index)

#     # Non-zero frequency in last two timepoints
#     last_two_cols = [str(int_tp - 1), str(int_tp)]
#     keep_gts_crit2 = set(fq_df[fq_df[last_two_cols].astype(bool).apply(sum, axis = 'columns') >= 2].index)

    # Rising in last two timepoints
    crit = (fq_df[str(int_tp)] - fq_df[str(int_tp - 1)] > 0)
    keep_gts_crit3 = set(fq_df[crit].index)
    
    # Above 3% frequency in last timepoint
    last_timepoint_min_fq = 0.03
    keep_gts_crit4 = set(fq_df[fq_df[str(int_tp)] >= last_timepoint_min_fq].index)
             
    keep_gts = keep_gts_crit3 & keep_gts_crit4
#     keep_gts = keep_gts_crit1 & keep_gts_crit2 & keep_gts_crit3 & keep_gts_crit4
    crit = (fitness_df['Genotype'].isin(keep_gts))
    dfs = fitness_df[crit]

    # Label most frequent gt as rising star if fq < 0.50 and fq > 0.50 later
    if obs_is_rising and obsnot_consensus:
        dfs['Rising star, observed'] = (dfs['Fitness'] >= obs_consensus_fitness)
    else:
        dfs['Rising star, observed'] = (dfs['Fitness'] > obs_consensus_fitness)

    print(row['data_risingstar_num'])
    display(dfs[dfs['Rising star, observed']])

5


/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Fitness,"Rising star, observed"
87,V..................,0.308412,True


6


/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Fitness,"Rising star, observed"


33


/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Genotype,Fitness,"Rising star, observed"
